In [1]:
%matplotlib inline
import nivapy3 as nivapy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import critical_loads as cl
import warnings

warnings.simplefilter('ignore')
plt.style.use('ggplot')

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(database='critical_loads')

Connection successful.


# Critical loads: Vegetation workflow (high-resolution method; 2018 onwards)

In Spring 2018, the worflow for calculating critical loads was refined to make use of new, higher-resolution input datasets. During November 2018, data handling for the Critical Loads project was also redesigned, with the ultimate aim of centralising all key datasets on NIVA's new cloud platform. 

This notebook deals with data processing for the vegetation exceedance calculations, using the new 0.1 degree deposition grid and raster-based vegetation dataset. A brief description of the old vegetation datasets is also provided.

**Note:** Deposition data supplied prior to 2017-18 use a different spatial grid (the "BLR" grid), and prior to 2012 a different (vector-based) vegetation dataset was also used. **The workflow here only applies to data supplied from 2017-18 onwards**; earlier data is *not* compatible with the calculations in this notebook. 

## 1. Organising vegetation data on the DSToolkit

NIVA's JupyterHub includes a PostGIS database server capable of storing relational and vector geospatial datasets. I have created a database named `critical_loads` and, within this, a schema named `vegetation`. This schema contains the following tables:

 * **exceedance_stats_0_1deg_grid:** Non-spatial table containing the area and percent exceeded for each 0.1 degree grid cell
 
 * **exceedance_stats_blr_grid:** Non-spatial table containing the area and percent exceeded for each BLR grid cell
 
 * **exceedance_stats_land_class:** Non-spatial table containing the area and percent exceeded for each vegetation class in the raster vegetation dataset
 
 * **land_class_crit_lds:** Non-spatial table defining the critical loads for each vegetation class
 
 * **vector_veg_pre_2012:** The old vector vegetation dataset used prior to 2012

## 2. Define data series and resolution of interest

Choose the dataset you whish to work with (see notebook 01 for a list of available `series_ids`.

**Note:** If you want to work with the 30 m resolution data, you will need to **log-in on one of the high memory machines**, as the grids involved are large.

In [3]:
# Series of interest
ser_id = 28

# Choose cell size (30m or 60m)
cell_size = 60

## 3. Create raster of critical loads for vegetation (optional)

The code below reads the raster vegetation data and reclassifies it based on a table of critical loads. It is not necessary to re-run this step unless the critical loads for vegetation are changed. 

**Note:** The function `cl.reclassify_raster()` requires specification of a `mask_tif` to define the land. This is because the vegetation data includes offshore regions classified as `'water'`, and we do not wish to include these in the CL calculations. The `shared` drive includes two possible mask grids (30 m and 60 m), both based on the extent of the old BLR grid.

### 3.1. Extract table of critical loads (optional)

In [4]:
## Read reclassification table from db
#sql = ("SELECT * FROM vegetation.land_class_crit_lds")
#rc_df = pd.read_sql(sql, eng)
#
## Set land class code as index
#rc_df.index = rc_df['norut_code']
#del rc_df['norut_code']
#
#rc_df

,norut_veg_class,eunis_code,eunis_veg_class,comment,cl_meq_m2_yr,cl_100smgn_m2_yr,cl_mgn_m2_yr
norut_code,,,,,,,
1,Barskog - tett tresjikt,G3,Coniferous woodland,None,35.714286,5,500
2,Barskog og blandingsskog - åpent tresjikt,G4.2,Mixed taiga woodland with Betula,None,35.714286,5,500
3,Lavrik furuskog,G3.B,Pine taiga woodland,None,35.714286,5,500
4,Lågurtskog og edellauvskog,G1,Broadleaved deciduous woodland,None,71.428571,10,1000
5,Høgstaude- og storbregnelauvskog,G1,Broadleaved deciduous woodland,None,71.428571,10,1000
6,Blåbær- og småbregnebjørkeskog,G1.918,Eurasian boreal Betula woods,None,35.714286,5,500
7,Kreklingbjørkeskog,G1.918,Eurasian boreal Betula woods,None,35.714286,5,500
8,Lavrik bjørkeskog,G1.918,Eurasian boreal Betula woods,None,35.714286,5,500
9,Tuemyr og lågvokst fastmattemyr,D1,Raised- and blanket bogs,None,35.714286,5,500


### 3.2. Reclassify vegetation dataset (optional)

In [6]:
# Reclassify
mask_tif = f'/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif'
veg_tif = f'/home/jovyan/shared/critical_loads/raster/sat_veg_{cell_size}m_norut.tif'
rc_tif = f'/home/jovyan/shared/critical_loads/raster/sat_veg_{cell_size}m_cl_100smgn_m2_yr.tif'
cl.reclassify_raster(veg_tif,
                     mask_tif,
                     rc_tif,
                     rc_df,
                     'cl_100smgn_m2_yr',
                     255)

## 4. Exceedances for vegetation

Exceedances for vegatation are calculated as follows:

 1. Read the raster of N deposition, which must already exist with the same properties (extent, cell size etc.) as the critical loads grid of interest here - see notebook 01
 
 2. Calculate the exceedance for each pixel, $i$, as $E_i = D_i - C_i$ (where $E$ is exceedance, $D$ is deposition and $C$ is the critical load)
 
 3. Save the exceedance grid as a GeoTiff. Calculate the total exceeded area for Norway as a whole, the proportion exceeded for each land class, and the proportion exceeded in each 0.1 degree vector cell
 
 4. Write summary results back to the database

In [ ]:
# CL raster (created in Section 3.2)
cl_tif = f'/home/jovyan/shared/critical_loads/raster/sat_veg_{cell_size}m_cl_100smgn_m2_yr.tif'

# N deposition raster (created in notebook 01)
ndep_tif = f'/home/jovyan/shared/critical_loads/raster/deposition/ndep_12-16_{cell_size}m.tif'

# Output exceedance .tifs to create
ex_tif = f'/home/jovyan/shared/critical_loads/raster/exceedance/veg_ex_12-16_{cell_size}m.tif'
ex_tif_bool = f'/home/jovyan/shared/critical_loads/raster/exceedance/veg_ex_12-16_{cell_size}m_bool.tif'

In [ ]:
#%%time
#
## Calculate exceedance
#ex_df = cl.calc_vegetation_exceedance_0_1deg(ndep_tif, cl_tif, ex_tif, ex_tif_bool, ser_id)
#
#ex_df

**If you connected to the database with `admin=True`**, these results can be saved back to the database.

In [ ]:
## Write summary data to db
#ex_df.to_sql('national_summary', 
#             eng,
#             'summaries',
#             if_exists='append',
#             index=False)

## 5. Zonal statistics

To save results from the cells below, **you must be connected to the database with `admin=True`**. Otherwise, set `write_to_db=False` to read simply read the data.

### 5.1. Exceedance per 0.1 degree grid cell

The code below calculates exceedance summary statistics for each 0.1 degree grid cell.

In [ ]:
# Calculate zonal statistics
gdf = cl.exceedance_stats_per_0_1deg_cell(ex_tif_bool, 
                                          ser_id, 
                                          eng, 
                                          categorical=True,
                                          category_map={1:'exceeded',
                                                        0:'not_exceeded'},
                                          nodata_value=255,
                                          write_to_db=False,
                                         )

gdf.head()

### 5.2. Exceedance per land use class

In [ ]:
# Calc exceedance per land cover class
veg_tif = f'/home/jovyan/shared/critical_loads/raster/sat_veg_{cell_size}m_all.tif'
df = cl.exceedance_stats_per_land_use_class(ex_tif_bool, 
                                            veg_tif, 
                                            ser_id,
                                            eng,
                                            nodata_value=255,
                                            write_to_db=False,
                                           )

df

### 5.3. Map exceedance per 0.1 degree grid square

In [ ]:
# Extract exceedance summaries
ex_gdf = cl.veg_exceedance_as_gdf_0_1deg(ser_id, eng)

# Choropleth map
ex_gdf.plot(column='pct_exceeded', 
            cmap='coolwarm',
            scheme='fisher_jenks',
            edgecolor=None,
            legend=True,
            legend_kwds={'loc':   'upper left',
                         'title': 'Vegetation exceedance (%)'},
            figsize=(8,8))

plt.axis('off')
plt.tight_layout()